# Pip install

In [ ]:
pip install pymilvus grpcio-tools milvus

# Restart Session before proceeding

In [ ]:
from pymilvus import Collection, connections

# from milvus import Milvus, DataType

from urllib.request import urlretrieve
import tarfile
import numpy as np
import struct
import numpy as np
import requests

# read_fbin

In [ ]:
def read_fbin(filename, start_idx=0, chunk_size=None):
    """ Read *.fbin file that contains float32 vectors
    Args:
        :param filename (str): path to *.fbin file
        :param start_idx (int): start reading vectors from this index
        :param chunk_size (int): number of vectors to read.
                                 If None, read all vectors
    Returns:
        Array of float32 vectors (numpy.ndarray)
    """
    with open(filename, "rb") as f:
        nvecs, dim = np.fromfile(f, count=2, dtype=np.int32)
        nvecs = (nvecs - start_idx) if chunk_size is None else chunk_size
        arr = np.fromfile(f, count=nvecs * dim, dtype=np.float32,
                          offset=start_idx * 4 * dim)
    return arr.reshape(nvecs, dim)

# Download dataset

In [ ]:
# Download dataset

# Text to Image 1B 1M
# link = "https://storage.yandexcloud.net/yandex-research/ann-datasets/T2I/base.1M.fbin"

# Deep1B 10M
link = 'https://storage.yandexcloud.net/yandex-research/ann-datasets/DEEP/base.10M.fbin'
f = requests.get(link, stream=True)
# f.text[:10]

In [ ]:
# Write the contents to a file

file_path = "base.10M.fbin"
with open(file_path, "wb") as myfile:
    myfile.write(f.content)

# Read dataset

In [ ]:
# Convert to np array

file_path = "base.10M.fbin"
my_dataset = read_fbin(file_path)
my_dataset.shape

(10000000, 96)

In [ ]:
my_dataset.shape[0]

10000000

# Connect to vector database

In [ ]:
TOKEN = '9ac70c2ae419bb92437b49afaee5e28a412b2b4a560366056e486c1e2b1f3af48f9a62a37551fa5735b5e4c27900abe963544dbf'
CLUSTER_ENDPOINT="https://in03-b29a7b5456b7b3f.api.gcp-us-west1.zillizcloud.com/"

connections.connect(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN)

# Upload to vector database

In [ ]:
# Choose collection in the database

collection = Collection("DeepDataset")

In [ ]:
n = my_dataset.shape[0]

list_batch_size   = 500000
upload_batch_size = 10000

for i in range(0, n, list_batch_size):

  indices = [i for i in range(i, i+list_batch_size)]

  my_list = my_dataset[i:i+list_batch_size].tolist()

  my_dict = [{'primary_key': indices[i], 'vector': my_list[i]} for i in range(list_batch_size)]

  for j in range(0, list_batch_size, upload_batch_size):
    # Extract a batch of entities
    batch_entities = my_dict[j:j+upload_batch_size]

    # Insert the batch into the collection
    mr = collection.insert(batch_entities)

ERROR:pymilvus.decorators:RPC error: [insert_rows], <MilvusException: (code=9, message=quota exceeded[reason=disk quota exceeded, please allocate more resources])>, <Time:{'RPC start': '2024-04-28 22:52:23.797704', 'RPC error': '2024-04-28 22:52:24.521006'}>


MilvusException: <MilvusException: (code=9, message=quota exceeded[reason=disk quota exceeded, please allocate more resources])>

In [ ]:
# my_dataset_tolist = my_dataset.tolist()

# my_dict = [{'primary_key': i, 'vector': my_dataset_tolist[i]} for i in range(len(my_dataset_tolist))]

In [ ]:
# # Upload 10k rows at a time

# total_rows = 1000000
# batch_size = 10000

# for i in range(0, total_rows, batch_size):
#     # Extract a batch of entities
#     batch_entities = my_dict[i:i+batch_size]

#     # Insert the batch into the collection
#     mr = collection.insert(batch_entities)